In [40]:
from copy import deepcopy
with open("input20.txt", "r") as f:
    lines = f.readlines()
    
def make_setsdicts(lines):
    flipflop_dict={}
    conj_dict={}
    for line in lines:
        if line[0]=="b":#broadcaster
            broadc_targets = line[line.find(">")+1:].replace(","," ").split()
        elif line[0]=="%":#flipflop
            flipflop_dict.update({line[1:line.find("-")-1]:[False,line[line.find(">")+1:].replace(","," ").split()]})    
        elif line[0]=="&":#conjunction
            conj_dict.update({line[1:line.find("-")-1]: [line[line.find(">")+1:].replace(","," ").split(), {}]}) #dictionary of inputs and last pulses empty
    for line in lines:#filling that dict
    #broadcast doesn't send to conjunction
        if line[0]=="%":#flipflop
            for dest in set(line[line.find(">")+1:].replace(","," ").split()):
                if dest in conj_dict:
                    conj_dict[dest][1].update({line[1:line.find("-")-1]:"low"})#all begin in "low" state
        elif line[0]=="&":#conjunction
            for dest in set(line[line.find(">")+1:].replace(","," ").split()):
                if dest in conj_dict:
                    conj_dict[dest][1].update({line[1:line.find("-")-1]:"low"})#all begin in "low" state

    return broadc_targets, flipflop_dict, conj_dict

def broadcast():
    [pulses.append(("broadcast", "low", dest)) for dest in broadc_targets]
    return None

def flipflop(pulsekind, module): #pulsekind = low/high
    if module not in flipflop_dict:
        raise ValueError("Flipflop attempted on non-flipflop module")
    if pulsekind=="high":
        return None
    flipflop_dict[module][0] = not flipflop_dict[module][0]
    [new_pulses.append((module,"high" if flipflop_dict[module][0] else "low", dest)) for dest in flipflop_dict[module][1]]
    return None

def conjunction(origin, pulsekind, module):
    conj_dict[module][1][origin] = pulsekind
    if set(conj_dict[module][1].values())=={"high"}:
        [new_pulses.append((module,"low", dest)) for dest in conj_dict[module][0]]
        return None
    [new_pulses.append((module,"high", dest)) for dest in conj_dict[module][0]]
    return None

broadc_targets, flipflop_dict, conj_dict = make_setsdicts(lines)
# dict_history = [(deepcopy(flipflop_dict), deepcopy(conj_dict))] #there is no cycle within 1000, storing this is useless
low_c, high_c = 0, 0

pulses = [] #format (origin, low/high, dest)
for i in range(1000):
    low_c +=1 #low pulse from button to broadcast
    broadcast()
    low_c += len(broadc_targets)
    new_pulses = ["placeholder"]
    while new_pulses != []:
        new_pulses=[]
        while pulses != []:
            origin, pulsekind, dest = pulses.pop(0)    
            if dest in flipflop_dict:
                flipflop(pulsekind, dest)
            elif dest in conj_dict:
                conjunction(origin, pulsekind, dest)
        # print(new_pulses)
        low_c += len([pulse for pulse in new_pulses if pulse[1]=="low"]) #must be list, sometimes same pulse twice in new_pulses
        high_c += len([pulse for pulse in new_pulses if pulse[1]=="high"]) #must be list, sometimes same pulse twice in new_pulses
        pulses = new_pulses.copy()
    # if (flipflop_dict, conj_dict) in dict_history:
    #     cycle_begin = dict_history.index((flipflop_dict, conj_dict))
    #     cycle_last = i
    #     break    
    # dict_history.append((deepcopy(flipflop_dict), deepcopy(conj_dict)))
    
low_c*high_c

812609846

In [69]:
from copy import deepcopy
import math

def cycle_run(lines):

    def make_setsdicts(lines):
        flipflop_dict={}
        conj_dict={}
        for line in lines:
            if line[0]=="b":#broadcaster
                broadc_targets = line[line.find(">")+1:].replace(","," ").split()
            elif line[0]=="%":#flipflop
                flipflop_dict.update({line[1:line.find("-")-1]:[False,line[line.find(">")+1:].replace(","," ").split()]})    
            elif line[0]=="&":#conjunction
                conj_dict.update({line[1:line.find("-")-1]: [line[line.find(">")+1:].replace(","," ").split(), {}]}) #dictionary of inputs and last pulses empty
        for line in lines:#filling that dict
        #broadcast doesn't send to conjunction
            if line[0]=="%":#flipflop
                for dest in set(line[line.find(">")+1:].replace(","," ").split()):
                    if dest in conj_dict:
                        conj_dict[dest][1].update({line[1:line.find("-")-1]:"low"})#all begin in "low" state
            elif line[0]=="&":#conjunction
                for dest in set(line[line.find(">")+1:].replace(","," ").split()):
                    if dest in conj_dict:
                        conj_dict[dest][1].update({line[1:line.find("-")-1]:"low"})#all begin in "low" state

        return broadc_targets, flipflop_dict, conj_dict

    def broadcast():
        [pulses.append(("broadcast", "low", dest)) for dest in broadc_targets]
        return None

    def flipflop(pulsekind, module): #pulsekind = low/high
        if module not in flipflop_dict:
            raise ValueError("Flipflop attempted on non-flipflop module")
        if pulsekind=="high":
            return None
        flipflop_dict[module][0] = not flipflop_dict[module][0]
        [new_pulses.append((module,"high" if flipflop_dict[module][0] else "low", dest)) for dest in flipflop_dict[module][1]]
        return None

    def conjunction(origin, pulsekind, module):
        conj_dict[module][1][origin] = pulsekind
        if set(conj_dict[module][1].values())=={"high"}:
            [new_pulses.append((module,"low", dest)) for dest in conj_dict[module][0]]
            return None
        [new_pulses.append((module,"high", dest)) for dest in conj_dict[module][0]]
        return None

    broadc_targets, flipflop_dict, conj_dict = make_setsdicts(lines)
    four_prev_conj = list(conj_dict['hp'][1].keys()) #knowing that 'hp' is the only one that points to output rx
    cycle_len = {mod:0 for mod in four_prev_conj}
    pulses = [] #format (origin, low/high, dest)
    for i in range(1,10**10):
        broadcast()
        new_pulses = ["placeholder"]
        while new_pulses != []:
            new_pulses=[]
            while pulses != []:
                origin, pulsekind, dest = pulses.pop(0)    
                if dest in flipflop_dict:
                    flipflop(pulsekind, dest)
                elif dest in conj_dict:
                    conjunction(origin, pulsekind, dest)
            pulses = new_pulses.copy()
            target_in_new_pulses = [(mod, 'high', 'hp') for mod in four_prev_conj if (mod, 'high', 'hp') in new_pulses]
            if target_in_new_pulses!=[]:
                for t in target_in_new_pulses:
                    cycle_len[t[0]] = i if cycle_len[t[0]]==0 else cycle_len[t[0]]
                if 0 not in cycle_len.values():
                    return cycle_len
    raise ValueError("Couldn't find cycle after 10**10 button presses")

with open("input20.txt", "r") as f:
    lines = f.readlines()

cycle_len = cycle_run(lines)
math.lcm(*cycle_len.values()), cycle_len

(245114020323037, {'sr': 3923, 'sn': 3967, 'rf': 4021, 'vq': 3917})

This solution takes some analyzing the circuit. One can see that only &hp signals to output rx. Furthermore, it also has 4 conjunctors leading to it. So the solution can be found finding the cycle of each conjunctor (knowing it operates in cycles!). The function finds the cycle length of each conjunctor by looking for the first time it activates hp with a high signal. A LCM gives the cycle when all 4 do it at the same time.

Helpful image: https://www.reddit.com/r/adventofcode/comments/18mypla/2023_day_20_input_data_plot/